# Function Testing

In [12]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from pandas import DataFrame
from patsy import dmatrices
from patsy import dmatrix
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import matplotlib.pyplot as plt
import matplotlib
import sys

In [13]:
from time import time
from datetime import date, timedelta
td = timedelta(days=1)

import os
os.environ['QT_API'] = 'pyside'
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [14]:
all_info = pd.read_csv('160205_cluster/integrated_0or1_larger_sect_with_cluster.csv', encoding='utf-8-sig')
mania_chart = pd.read_csv('160130/mania_data_revised.csv', encoding='utf-8-sig')
freq_artists = pd.read_csv('160204_advanced/freqArtists.csv', encoding='utf-8-sig')
freq_songs = pd.read_csv('160204_advanced/freqSongs.csv', encoding='utf-8-sig')

In [31]:
all_info = pd.read_csv('160205_cluster/integrated_0or1_larger_sect_with_cluster.csv', encoding='utf-8-sig')
freq_songs = pd.read_csv('160204_advanced/freqSongs.csv', encoding='utf-8-sig')

normal_info = DataFrame({})

df = all_info.copy()
df = df.select_dtypes(exclude=['object'])
#df.events
normal_info = ( df - all_info.mean() ) / all_info.std()
normal_info = normal_info[['meanT', 'maxT', 'minT', 'dewPoint',
                           'avgH', 'maxH', 'minH', 'avgW',
                           'maxW', 'visibleD', 'pressure', 'wind_chill',
                           'heat_index', 'discomfort_index']] # 14 column

all_info.drop(all_info.columns[[1,2,3,4,5,6,7,8,9,10,11,14,15,16]], axis=1, inplace=True)

# Get Logistic data & Join column
month_logit = pd.get_dummies(all_info['Month'], prefix='month') # 12 column
season_logit = pd.get_dummies(all_info['Season'], prefix='season') # 4 column
cluster_logit = pd.get_dummies(all_info['cluster'], prefix='cluster') # 8 column

print all_info.shape
all_info = all_info.join(month_logit)
all_info = all_info.join(season_logit)
all_info = all_info.join(normal_info)
all_info = all_info.join(cluster_logit)
all_info['cnt'] = 1
print all_info.shape
all_info

(1458, 982)
(1458, 1021)


,Date,events,highlight,Title.ID1,Title.ID2,Title.ID3,Title.ID4,Title.ID5,Title.ID6,Title.ID7,...,discomfort_index,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cnt
0,2012-01-01,Snow,"Waxing Crescent, 49",0,0,1,0,0,0,0,...,-1.612513,0,0,0,0,0,0,1,0,1
1,2012-01-02,None,"First Quarter, 58",0,0,1,0,0,0,0,...,-1.695519,1,0,0,0,0,0,0,0,1
2,2012-01-03,"Rain,Snow","Waxing Gibbous, 67",0,0,0,0,0,0,0,...,-1.553913,0,0,0,0,0,0,1,0,1
3,2012-01-04,None,"Waxing Gibbous, 76",0,0,0,0,0,0,0,...,-1.757752,1,0,0,0,0,0,0,0,1
4,2012-01-05,None,"Waxing Gibbous, 83",0,0,0,0,0,0,0,...,-1.616854,1,0,0,0,0,0,0,0,1
5,2012-01-06,None,"Waxing Gibbous, 90",0,0,0,0,0,0,0,...,-1.357040,1,0,0,0,0,0,0,0,1
6,2012-01-07,None,"Waxing Gibbous, 95",0,0,0,0,0,0,0,...,-1.434077,1,0,0,0,0,0,0,0,1
7,2012-01-08,None,"Waxing Gibbous, 98",0,0,1,0,0,0,0,...,-1.502555,1,0,0,0,0,0,0,0,1
8,2012-01-09,None,"Full, 100",0,0,0,0,0,0,0,...,-1.442682,1,0,0,0,0,0,0,0,1
9,2012-01-10,None,"Full, 99",0,0,0,0,0,0,0,...,-1.363451,1,0,0,0,0,0,0,0,1


In [32]:
target_info = []
target_chart = []
target_date = date(2012, 1, 1)
for i in range(1461):
    target_date_str = target_date.strftime('%Y-%m-%d')
    target_date_int = target_date.year * 10000 + target_date.month * 100 + target_date.day
    
    target_info.append(all_info[all_info.Date == target_date_str])
    target_chart.append(mania_chart[mania_chart.Date == target_date_int])
    target_date = target_date + td

In [63]:
target_info[0].columns[-23:-1]
# all_info.columns[3]
features_NORMAL = 'meanT + maxT + minT + dewPoint + avgH + maxH + minH + avgW + maxW + visibleD + pressure + wind_chill + heat_index + discomfort_index'

Index([           u'meanT',             u'maxT',             u'minT',
               u'dewPoint',             u'avgH',             u'maxH',
                   u'minH',             u'avgW',             u'maxW',
               u'visibleD',         u'pressure',       u'wind_chill',
             u'heat_index', u'discomfort_index',        u'cluster_1',
              u'cluster_2',        u'cluster_3',        u'cluster_4',
              u'cluster_5',        u'cluster_6',        u'cluster_7',
              u'cluster_8'],
      dtype='object')

In [69]:
def testing(features = features):
    jc = {}
    JaccardDF = DataFrame(jc)
    JC_fm_lst = []
    JC_target_lst = []
    intersect_lst = []
    cur_date_lst = []
    origin_correct_lst = []
    correct_lst = []
    JC_OnL_lst = []
    
    ## 독립변수에서 feature 뽑아내기
    y = all_info['Title.ID1']
    ainfo_train, ainfo_test, y_train, y_test = train_test_split(all_info, y, test_size=0.3, random_state=0)

    X_train = dmatrix(features, ainfo_train, return_type="dataframe")
    X_test = dmatrix(features, ainfo_test, return_type="dataframe")
    y_train = ainfo_train[ainfo_train.columns[[range(3,631)]]]
    y_test = ainfo_test[ainfo_test.columns[[range(3,631)]]]

    cur_date = date(2012, 1, 1)
    fm = freq_songs[:100]['Title']

    model_lst = []

    ## 개별 곡 모델 트레이닝
    for i in range(628):
        model = LogisticRegression()
        model.fit(X_train, y_train[y_train.columns[i]])
    #     test_score_lst.append(model_lst[i-1].predict_proba(X_test)[:,1])
        model_lst.append(model)

    for j in range(1461):
        try:
            target_data = ainfo_test[ainfo_test['Date'] == target_info[j]['Date'].get_values()[0]]
            if not target_data.shape[0]:
                cur_date = cur_date + td
                continue

            target_score_lst = []
            name_lst = []
            id_lst = []
            temp = {}
            lrdf = DataFrame(temp)

            tarX = dmatrix(features, target_info[j], return_type="dataframe")

            for i in range(628):
        #             train_score_lst.append(model2.predict_proba(X_train)[:,1])
        #             test_score_lst.append(model_lst[i-1].predict_proba(X_test)[:,1])
                target_score_lst.append(model_lst[i].predict_proba(tarX)[:,1][0])
                id_lst.append(i)

                # generate evaluation metrics
                title_name = freq_songs.loc[i].get_value('Title')
                name_lst.append(title_name)

            lrdf['Score(target)'] = target_score_lst
            lrdf['Title'] = name_lst
            lrdf['ID'] = id_lst

            target_rating = lrdf.sort_values('Score(target)', ascending=False)[:100]

            target_chart[j]['name'] = target_chart[j].Title + '%by%' + target_chart[j].Artist

            t2 = target_chart[j]['name']
            t1 = target_rating['Title']
            t2.columns = ['Title']
            #t3 = t1.append(t2)
            #t3 = t1.intersection(t2)
            t_intersect = pd.Series(list(set(t1) & set(t2)))
            t_union = pd.Series(list(set(t1) | set(t2)))
            JC_target = t_intersect.count() / float(t_union.count())

            fm_intersect = pd.Series(list(set(fm) & set(t2)))
            fm_union = pd.Series(list(set(fm) | set(t2)))
            JC_fm = fm_intersect.count() / float(fm_union.count())

            last_intersect = pd.Series(list(set(t_intersect) & set(fm_intersect)))
            last_union = pd.Series(list(set(fm_intersect) | set(t_intersect)))
            JC_last = last_intersect.count() / float(last_union.count())
#             print str(cur_date) + '==============='
#             print 'Recomm Correct: ', t_intersect.count()
#             print JC_fm, JC_target
#             #print 'Original Rank ============'
#             #print fm_intersect
#             #print 'Regression Rank ============'
#             #print t_intersect
#             print 'cross ========'
#             print JC_last
        #     print last_intersect
            JC_fm_lst.append(JC_fm)
            JC_target_lst.append(JC_target)
            intersect_lst.append(last_intersect.count())
            origin_correct_lst.append(fm_intersect.count())
            JC_OnL_lst.append(JC_last)
            correct_lst.append(t_intersect.count())
            cur_date_lst.append(str(cur_date))

            cur_date = cur_date + td
        except IndexError as e:
            print "There is not " + str(cur_date)
            cur_date = cur_date + td
            continue

    JaccardDF['JC_Original'] = JC_fm_lst
    JaccardDF['JC_Logit'] = JC_target_lst
    JaccardDF['O_Correct'] = origin_correct_lst
    JaccardDF['L_Correct'] = correct_lst
    JaccardDF['JC_OnL'] = JC_OnL_lst
    JaccardDF['noIntersect'] = intersect_lst
    JaccardDF['Date'] = cur_date_lst
    return JaccardDF

In [95]:
features_ALLINONE = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
            + season_spring + season_summer + season_fall + season_winter + heat_index + wind_chill\
            + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
features_CLUSTER = 'cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
features_NORMAL = 'meanT + maxT + minT + dewPoint + avgH + maxH + minH + avgW + maxW + pressure + wind_chill + heat_index + discomfort_index'
features_DISCRETE = 'T0 + T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8 + T9 + H0 + H1 + H2 + H3 + H4 + H5 + H6 + H7 + H8 + H9 + W0 + W1\
            + season_spring + season_summer + season_fall + season_winter + heat_index + wind_chill'
features_NORMALwCLUSTER = 'maxT + maxH + maxW\
            + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
f1 = 'season_spring + season_summer + season_fall + season_winter + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
f2 = 'maxT + maxH + maxW + season_spring + season_summer + season_fall + season_winter'
f3 = 'meanT + avgH + avgW + season_spring + season_summer + season_fall + season_winter'
f4 = 'maxT + maxH + maxW + season_spring + season_summer + season_fall + season_winter\
        + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
f5 = 'maxT + maxH + maxW + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
f6 = 'meanT + avgH + avgW\
            + season_spring + season_summer + season_fall + season_winter + heat_index + wind_chill\
            + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'
f7 = 'maxT + maxH + maxW\
            + season_spring + season_summer + season_fall + season_winter + heat_index + wind_chill\
            + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8'


# features = 'cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6 + cluster_7 + cluster_8\
#              + heat_index + wind_chill'


In [96]:
# lr_all = testing(features_ALLINONE)
# lr_cluster = testing(features=features_CLUSTER)
# lr_normal = testing(features=features_NORMAL)
# lr_discrete = testing(features=features_DISCRETE)
# lr_normalWcluster = testing(features_NORMALwCLUSTER)
lr1 = testing(f1)
lr2 = testing(f2)
lr3 = testing(f3)
lr4 = testing(f4)
lr5 = testing(f5)
lr6 = testing(f6)
lr7 = testing(f7)

There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17
There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17
There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17
There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17
There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17
There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17
There is not 2012-11-15
There is not 2013-03-16
There is not 2013-08-17


C:\Users\jinhyeok\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [94]:
lr_normalWcluster.describe()

,JC_Original,JC_Logit,O_Correct,L_Correct,JC_OnL,noIntersect
count,438.000000,438.000000,438.000000,438.000000,438.000000,438.000000
mean,0.071809,0.073817,13.280822,13.607306,0.536472,9.347032
std,0.025398,0.027979,4.384928,4.795225,0.156665,3.726721
min,0.015228,0.010101,3.000000,2.000000,0.000000,0.000000
25%,0.053191,0.052632,10.000000,10.000000,0.437500,7.000000
50%,0.069519,0.069519,13.000000,13.000000,0.533333,9.000000
75%,0.086957,0.092896,16.000000,17.000000,0.636364,12.000000
max,0.150289,0.185629,26.000000,31.000000,1.000000,24.000000


In [80]:
plotDF = DataFrame({})
plotDF['BaseLine'] = lr_all['JC_Original']
plotDF['LR_ALLINONE'] = lr_all['JC_Logit']
plotDF['LR_CLUSTER'] = lr_cluster['JC_Logit']
plotDF['LR_NORMAL'] =  lr_normal['JC_Logit']
plotDF['LR_DISCRETE'] = lr_discrete['JC_Logit']
plotDF.mean()

BaseLine    0.071809
ALLINONE    0.087051
CLUSTER     0.072450
NORMAL      0.077603
DISCRETE    0.087382
dtype: float64

In [83]:
plotDF.mean().plot(kind='bar')

In [ ]:
lr_all['JC_Original'].plot(kind=bar)
lr_all['JC_Logit'].plot()
lr_cluster['JC_Logit'].plot()
lr_normal['JC_Logit'].plot()
lr_discrete['JC_Logit'].plot()